In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-data/climate_data.csv


# import datbase

In [77]:
import sklearn
rain_df= pd.read_csv('/kaggle/input/climate-data/climate_data.csv')
rain_df.drop(['date','station_id'],inplace=True, axis=1)

rain_df

,Tn,Tx,Tavg,RH_avg,RR,ss,ff_x,ddd_x,ff_avg,ddd_car
0,21.4,30.2,27.1,82.0,9.0,0.5,7.0,90.0,5.0,E
1,21.0,29.6,25.7,95.0,24.0,0.2,6.0,90.0,4.0,E
2,20.2,26.8,24.5,98.0,63.0,0.0,5.0,90.0,4.0,E
3,21.0,29.2,25.8,90.0,0.0,0.1,4.0,225.0,3.0,SW
4,21.2,30.0,26.7,90.0,2.0,0.4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
589260,25.2,31.2,29.2,74.0,0.0,1.4,4.0,280.0,2.0,C
589261,25.3,31.6,28.1,78.0,NaN,3.0,12.0,260.0,2.0,C
589262,24.6,32.3,28.4,81.0,NaN,6.5,5.0,260.0,2.0,SW
589263,25.2,32.6,28.4,80.0,0.0,2.4,7.0,260.0,2.0,C


# **Data Inspection**

In [78]:
print("Shape of the data frame: ",rain_df.shape)
print("Total null values: ",rain_df.isna().sum())
print("Duplicate values: ",rain_df.duplicated().sum())

Shape of the data frame:  (589265, 10)
Total null values:  Tn          23383
Tx          37736
Tavg        45105
RH_avg      48182
RR         125384
ss          43721
ff_x        10214
ddd_x       13128
ff_avg      10127
ddd_car     13739
dtype: int64
Duplicate values:  1838


# **remove duplicates and making the target values be 0 or 1**

In [79]:
rain_df["RR"] = rain_df["RR"].apply(lambda x: 1 if x > 0 else 0)
rain_df.drop_duplicates(inplace=True)
print("Shape of the data frame: ",rain_df.shape)
print("\n")
print("Species categories with its count \n",rain_df["RR"].value_counts())
print("Duplicate values: ",rain_df.duplicated().sum())

Shape of the data frame:  (586556, 10)


Species categories with its count 
 RR
0    313723
1    272833
Name: count, dtype: int64
Duplicate values:  0


# Encode categorical variables

In [80]:
categorical_columns = ['ddd_car']
rain_df = pd.get_dummies(rain_df, columns=categorical_columns, drop_first=True)

# **spliting dataset into features and label**

In [81]:
X=rain_df.drop(["RR"],axis=1)
Y=rain_df["RR"]

# replace missing values with mean

In [82]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# spliting dataset into train data and test data

In [83]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_imputed,Y,test_size=0.2,random_state=3)

# Using Logistic Regression

In [84]:
from sklearn.linear_model import LogisticRegression
logistic_reg=LogisticRegression()
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
logistic_reg.fit(X_train,Y_train)
predicted_result=logistic_reg.predict(X_test)
print("Accuracy Score: ",accuracy_score(Y_test, predicted_result))

print("Confusion Matrix:\n ",confusion_matrix(Y_test, predicted_result))

print("Classification Report:\n ",classification_report(Y_test, predicted_result))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Score:  0.6895884478996182
Confusion Matrix:
  [[45795 16773]
 [19642 35102]]
Classification Report:
                precision    recall  f1-score   support

           0       0.70      0.73      0.72     62568
           1       0.68      0.64      0.66     54744

    accuracy                           0.69    117312
   macro avg       0.69      0.69      0.69    117312
weighted avg       0.69      0.69      0.69    117312



# Using KNN

In [85]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=7)  
knn_model.fit(X_train, Y_train)
knn_predicted_result = knn_model.predict(X_test)

print("KNN Accuracy Score: ", accuracy_score(Y_test, knn_predicted_result))
print("KNN Confusion Matrix:\n", confusion_matrix(Y_test, knn_predicted_result))
print("KNN Classification Report:\n", classification_report(Y_test, knn_predicted_result))

KNN Accuracy Score:  0.6791973540643753
KNN Confusion Matrix:
 [[42989 19579]
 [18055 36689]]
KNN Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.69      0.70     62568
           1       0.65      0.67      0.66     54744

    accuracy                           0.68    117312
   macro avg       0.68      0.68      0.68    117312
weighted avg       0.68      0.68      0.68    117312



# Using K-means

In [86]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=2, random_state=42)  
X_clusters = kmeans_model.fit_predict(X_imputed)


kmeans_labels = np.zeros_like(Y)
kmeans_labels[X_clusters == 1] = 1 

print("K-Means Accuracy Score: ", accuracy_score(Y, kmeans_labels))
print("K-Means Confusion Matrix:\n", confusion_matrix(Y, kmeans_labels))
print("K-Means Classification Report:\n", classification_report(Y, kmeans_labels))

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


K-Means Accuracy Score:  0.556315168543157
K-Means Confusion Matrix:
 [[182687 131036]
 [129210 143623]]
K-Means Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.58      0.58    313723
           1       0.52      0.53      0.52    272833

    accuracy                           0.56    586556
   macro avg       0.55      0.55      0.55    586556
weighted avg       0.56      0.56      0.56    586556



# Using RandomForest

In [87]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust hyperparameters as needed
random_forest.fit(X_train, Y_train)
rf_predicted_result = random_forest.predict(X_test)

print("Random Forest Accuracy Score: ", accuracy_score(Y_test, rf_predicted_result))
print("Random Forest Confusion Matrix:\n", confusion_matrix(Y_test, rf_predicted_result))
print("Random Forest Classification Report:\n", classification_report(Y_test, rf_predicted_result))

Random Forest Accuracy Score:  0.7029289416257501
Random Forest Confusion Matrix:
 [[45073 17495]
 [17355 37389]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.72      0.72     62568
           1       0.68      0.68      0.68     54744

    accuracy                           0.70    117312
   macro avg       0.70      0.70      0.70    117312
weighted avg       0.70      0.70      0.70    117312



# Using Decision Tree

In [88]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=42)  # You can adjust hyperparameters as needed
decision_tree.fit(X_train, Y_train)
dt_predicted_result = decision_tree.predict(X_test)

print("Decision Tree Accuracy Score: ", accuracy_score(Y_test, dt_predicted_result))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(Y_test, dt_predicted_result))
print("Decision Tree Classification Report:\n", classification_report(Y_test, dt_predicted_result))

Decision Tree Accuracy Score:  0.6220761729405346
Decision Tree Confusion Matrix:
 [[40347 22221]
 [22114 32630]]
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.64      0.65     62568
           1       0.59      0.60      0.60     54744

    accuracy                           0.62    117312
   macro avg       0.62      0.62      0.62    117312
weighted avg       0.62      0.62      0.62    117312



# Using Naive Bayes

In [89]:
from sklearn.naive_bayes import GaussianNB
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, Y_train)
nb_predicted_result = naive_bayes.predict(X_test)

print("Naive Bayes Accuracy Score: ", accuracy_score(Y_test, nb_predicted_result))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(Y_test, nb_predicted_result))
print("Naive Bayes Classification Report:\n", classification_report(Y_test, nb_predicted_result))

Naive Bayes Accuracy Score:  0.6292877114020731
Naive Bayes Confusion Matrix:
 [[35675 26893]
 [16596 38148]]
Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.57      0.62     62568
           1       0.59      0.70      0.64     54744

    accuracy                           0.63    117312
   macro avg       0.63      0.63      0.63    117312
weighted avg       0.64      0.63      0.63    117312

